In [16]:
pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [17]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv("chatgpt1.csv")
df['createdtime'] = pd.to_datetime(df['createdtime'])
df.fillna("", inplace=True)
df.shape

(50001, 20)

In [45]:
client = pymongo.MongoClient("mongodb+srv://<username>:<password>@demo.zwn4puw.mongodb.net/test")
db = client['twitter']

In [46]:
columns_map = {'Datetime' : 'createdTime', 'Text': 'tweetText', 'Username' : 'username', 
               'Permalink': 'permalinkUrl', 'User' : 'userUrl', 'Outlinks': 'outLink',
               'CountLinks': 'countLink', 'ReplyCount' : 'replyCount', 'RetweetCount': 'retweetCount', 
               'LikeCount': 'likeCount', 'QuoteCount': 'quoteCount', 'ConversationId': 'conversationId', 
               'Language': 'lang', 'Source': 'sourceUrl', 'Media': 'mediaDetails', 
               'QuotedTweet': 'quotedTweet', 'MentionedUsers': 'userInfo', 'hash_tag' : 'hashtag', 
               'hastag_counts': 'hastagCounts', 'Tweet Id': 'tweetId' }
df = df.rename(columns=columns_map)
df.fillna('', inplace=True)
df.columns

Index(['createdtime', 'Tweetid', 'Tweettext', 'username', 'permalinkUrl',
       'User_url', 'outLink', 'countLink', 'replyCount', 'retweetCount',
       'likeCount', 'quoteCount', 'conversationId', 'Lang', 'Source_url',
       'mediaDetails', 'quotedTweet', 'userInfo', 'hashtag', 'hastagCounts'],
      dtype='object')

In [21]:
#User Collection
def get_users(value):
    value = value.replace('[', '').replace(']', '').replace(' ', '').replace('\"', '').replace('\'', '')
    users = []
    for user in value.split(',User('):
        user = user.replace('User(', '').replace('(', '').replace(')', '') 
        users.append(user)
    return users

def get_user_name(info):
    return info.split(',')[0].split('username=')[1].replace('\'', '')
                                            
columns = ['username', 'User_url']
user_df = df[columns]
user_df = user_df.drop_duplicates()

xtra_user = df[['userInfo']].dropna().drop_duplicates()
xtra_user = xtra_user[xtra_user['userInfo'] != '']
xtra_user['userInfo'] = [get_users(x) for x in  xtra_user['userInfo']]
xtra_user = xtra_user.explode('userInfo')
xtra_user = xtra_user[xtra_user['userInfo'] != np.nan].drop_duplicates()
xtra_user = xtra_user.dropna().drop_duplicates()
xtra_user['username'] = [get_user_name(x) for x in xtra_user['userInfo']]
xtra_user['User_url'] = 'https://twitter.com/' + xtra_user['username']
xtra_user = xtra_user.drop_duplicates(subset=['username', 'User_url'], keep='last')
all_user_df = pd.merge(user_df, xtra_user, how="outer", on=["username", "User_url"]).drop_duplicates()
all_user_df.fillna("", inplace=True)
user_dict=all_user_df.to_dict("records")
collection = db['users']
collection.insert_many(user_dict)


In [47]:
# Tweet Collection
def get_source_name(x):
    return x.split(">")[1].split("<")[0].strip()
source_df = df[['Source_url']]
source_df['Used_source'] = [get_source_name(x) for x in df['Source_url']]
source_df=source_df['Used_source']
url_col=['permalinkUrl','outLink', 'countLink','quotedTweet']
url_df=df[url_col]
url_dict=url_df.to_dict("records")
columns = ['Tweetid','Tweettext','username','createdtime', 'Lang', 'conversationId', 'replyCount','retweetCount',
           'likeCount','quoteCount','hastagCounts']
tweet_df = df[columns]
tweet_df= pd.concat([tweet_df, source_df], axis=1)
tweet_df.head(5)
tweet_df['urls']=url_dict
tweet_dict=tweet_df.to_dict("records")
collection = db['tweets']
collection.insert_many(tweet_dict)


C:\Users\priya\AppData\Local\Temp\ipykernel_3788\827468822.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_df['Used_source'] = [get_source_name(x) for x in df['Source_url']]


In [25]:
#hashtag collection
def convert_to_list(value):
    return value.replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(",")
columns = ['Tweetid', 'hashtag']
hashtag_df = df[columns]

hashtag_df['hashtag'] = [convert_to_list(x) for x in  hashtag_df['hashtag']]

hashtag_df = hashtag_df.explode('hashtag')
hashtag_df = hashtag_df[hashtag_df['hashtag'] != '']
hashtag_dict =hashtag_df.to_dict("records")
collection = db['hashtags']
collection.insert_many(hashtag_dict)


C:\Users\priya\AppData\Local\Temp\ipykernel_3788\3249413257.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hashtag_df['hashtag'] = [convert_to_list(x) for x in  hashtag_df['hashtag']]


In [36]:
# media collection 

def clear_field(x):
    return x.replace('[', '').replace(']', '').strip()

def get_all_media(mediaDetails):
    x = mediaDetails.replace('[', '').replace(']', '').replace(' ', '')
    x = x.replace('Photo(', 'Photo(Photo(')
    x = x.replace('Video(', 'Video(Video(')
    x = x.replace('Gif(', 'Gif(Gif(')
    
    res = [x]
    for s in ['),Photo(', '), Video(', '), Gif(']:
        temp = [p for r in res for p in r.split(s)]
        res = temp
    
    result = []
    for r in res:
        r = r.replace('Photo(Photo(', 'Photo(')
        r = r.replace('Video(Video(', 'Video(')
        r = r.replace('Gif(Gif(', 'Gif(')
        if not r.endswith(')'):
            r = r+')'
        result.append(r)
    return result
    

def get_mediaType(media):
    return media.replace('[', '').replace(']', '').replace(' ', '').split('(')[0]


pd.set_option('display.max_colwidth', None)

columns = ['Tweetid', 'mediaDetails']
media_df = df[columns]
media_df['mediaDetails'] = [clear_field(x) for x in media_df['mediaDetails']]
media_df['mediaDetails'] = media_df['mediaDetails'].where(media_df['mediaDetails'] != '', np.nan)
media_df = media_df.dropna().drop_duplicates()


media_df['mediaDetails'] = [get_all_media(x) for x in media_df['mediaDetails']]
media_df = media_df.explode('mediaDetails')

media_df['mediaType'] = [get_mediaType(x) for x in media_df['mediaDetails']]
media_df_all=media_df[['Tweetid','mediaType','mediaDetails']]
media_df
media_dict =media_df_all.to_dict("records")
collection = db['media']
collection.insert_many(media_dict)


C:\Users\priya\AppData\Local\Temp\ipykernel_3788\1304118638.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  media_df['mediaDetails'] = [clear_field(x) for x in media_df['mediaDetails']]
C:\Users\priya\AppData\Local\Temp\ipykernel_3788\1304118638.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  media_df['mediaDetails'] = media_df['mediaDetails'].where(media_df['mediaDetails'] != '', np.nan)


,Tweetid,mediaDetails,mediaType
2,1617156308926340000,"Photo(previewUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnFLF3SWIAAtThd?format=jpg&name=large')",Photo
8,1617156404137290000,"Photo(previewUrl='https://pbs.twimg.com/media/FnFLLMhaEAAXeSF?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnFLLMhaEAAXeSF?format=jpg&name=large')",Photo
8,1617156404137290000,"Photo(previewUrl='https://pbs.twimg.com/media/FnFLLMjaUAUZ_-h?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnFLLMjaUAUZ_-h?format=jpg&name=large')",Photo
8,1617156404137290000,"Photo(previewUrl='https://pbs.twimg.com/media/FnFLLMkaYAAa5tF?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnFLLMkaYAAa5tF?format=jpg&name=large')",Photo
8,1617156404137290000,"Photo(previewUrl='https://pbs.twimg.com/media/FnFLLMiacAAttvJ?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnFLLMiacAAttvJ?format=jpg&name=large')",Photo
...,...,...,...
49997,1617778726393240000,"Photo(previewUrl='https://pbs.twimg.com/media/FnOBJywXEAE2X1c?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnOBJywXEAE2X1c?format=jpg&name=large')",Photo
49998,1617778728481990000,"Photo(previewUrl='https://pbs.twimg.com/media/FnOBLWWX0AU55Yl?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnOBLWWX0AU55Yl?format=jpg&name=large')",Photo
49998,1617778728481990000,"Photo(previewUrl='https://pbs.twimg.com/media/FnOBLWWWAAA7syy?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnOBLWWWAAA7syy?format=jpg&name=large')",Photo
49998,1617778728481990000,"Photo(previewUrl='https://pbs.twimg.com/media/FnOBLWTXoAA7ufW?format=jpg&name=small',fullUrl='https://pbs.twimg.com/media/FnOBLWTXoAA7ufW?format=jpg&name=large')",Photo


In [48]:
#user_mentions
def get_users(value):
    value = value.replace('[', '').replace(']', '').replace(' ', '').replace('\"', '').replace('\'', '')
    users = []
    for user in value.split(',User('):
        user = user.replace('User', '').replace('(', '').replace(')', '') 
        users.append(user)
    return users

def get_user_name(info):
    return info.split(',')[0].split('username=')[1].replace('\'', '')
                                            
def get_id(info):
    return info.split(',')[1].split('id=')[1].replace('\'', '')
def get_display_name(info):
    return info.split(',')[2].split('displayname=')[1].replace('\'', '')
                                            
                                            
columns = ['Tweetid', 'userInfo']
user_mentions_df = df[columns].drop_duplicates()

user_mentions_df['userInfo'] = [get_users(x) for x in  user_mentions_df['userInfo']]
user_mentions_df = user_mentions_df.explode('userInfo')
user_mentions_df = user_mentions_df[user_mentions_df['userInfo'] != '']
user_mentions_df['username'] = [get_user_name(x) for x in user_mentions_df['userInfo']]
user_mentions_df['userid'] = [get_id(x) for x in user_mentions_df['userInfo']]
user_mentions_df['displayname'] = [get_display_name(x) for x in user_mentions_df['userInfo']]
mentions_df = user_mentions_df[['Tweetid', 'username','userid','displayname']]
mentions_df.head(20)
usermentions_dict =mentions_df.to_dict("records")
collection = db['user_mentions']
collection.insert_many(usermentions_dict)
